### *Assignment 2*: Group Recommendations

In [1]:
from group_recommendation import GroupRecommendation
import pandas as pd

userGroup = [1, 50, 600]

**(a)** For producing group recommendation, we will use the user-based collaborative
filtering approach as this implemented in Assignment 1. Specifically, for producing group
recommendations, we will first compute the movies recommendations for each user in
the group, and then we will aggregate the lists of the individual users, so as to produce a
single list of movies for the group.

You will implement two well established aggregation methods for producing the group
recommendations.

The first aggregation approach is the average method. The main idea behind this
approach is that all members are considered equals. So, the rating of an item for a group
of users will be given be averaging the scores of an item across all group members.
The second aggregation method is the least misery method, where one member can act
as a veto for the rest of the group. In this case, the rating of an item for a group of users is
computed as the minimum score assigned to that item in all group members
recommendations.

Produce a group of 3 users, and for this group, show the top-10 recommendations, i.e.,
the 10 movies with the highest prediction scores that: 
1. the average method suggests,
2. the least misery method suggests. 


In [2]:
aggregation = GroupRecommendation.aggregate_users_recommendations(userGroup)

pd.set_option('max_colwidth', None)
pd.DataFrame(aggregation.items(), columns=['movieId', 'predictions'])

,movieId,predictions
0,3972,"[6.223522167487685, 4.63778801843318, 4.468753723340879]"
1,55052,"[6.110281749369218, 4.524547600314714, 1.5]"
2,3469,"[4.366379310344827, 2.7806451612903227, 4.562909567496724]"
3,913,"[6.252742946708464, 2.7806451612903227, 2.991480996068152]"
4,785,"[4.351966123970952, 3.1731496589191277, 4.515290519877675]"
5,5014,"[4.366379310344827, 4.841251221896384, 2.991480996068152]"
6,555,"[6.641379310344828, 3.187798093897363, 4.29074066865121]"
7,1204,"[3.1391065830721, 4.5, 4.710835834777829]"
8,2997,"[4.0, 4.413099281098879, 4.5]"
9,319,"[6.769157088122605, 3.2216641955263716, 3.2503045254799168]"


In [3]:
aggregation = GroupRecommendation.average_aggregation(userGroup)
pd.DataFrame(aggregation, columns=['movieId', 'avg predicition'])

,movieId,avg predicition
0,3972,5.110021
1,555,4.706639
2,30803,4.619085
3,5066,4.542103
4,42728,4.542103
5,1233,4.446583
6,319,4.413709
7,2997,4.304366
8,3567,4.166168
9,1204,4.116647


In [4]:
aggregation = GroupRecommendation.least_misery_aggregation(userGroup)
pd.DataFrame(aggregation, columns=['movieId', 'min predicition'])

,movieId,min predicition
0,3972,4.468754
1,2997,4.000000
2,319,3.221664
3,555,3.187798
4,785,3.173150
5,1204,3.139107
6,5014,2.991481
7,5066,2.991481
8,1616,2.991481
9,475,2.991481
